# Import Libraries

In [1]:
import os
import pandas as pd
import torch
import numpy as np
from Preprocess import Process
from NeuralNetworkDIY import ClassifierNetwork, Model
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from PIL import Image

In [2]:
print(torch.cuda.is_available())

True


# Parameters for the dataset

In [3]:
RAW_DIR = r"C:\Users\annal\ADL\archive\real-vs-fake"
PROCESSED_DIR = "data/processed"
IMAGE_SIZE=(128,128)

# Image Preprocessing

In [4]:
process=Process(RAW_DIR, PROCESSED_DIR, IMAGE_SIZE, delete_process_dir=True, batch_size=1000, aug_amount=0)
process.run(0.9, 0.1, 0)

PermissionError: [WinError 5] A hozzáférés megtagadva: 'data/processed\\128\\train\\rgb'

# DataLoader and Custom Dataset

In [4]:
class CustomImageDataset(Dataset):
    def __init__(self, directory):
        self.directory = directory
        self.image_paths = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith((".png", ".jpg", ".jpeg"))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("RGB")

        transform = transforms.ToTensor()
        
        img= transform(img)
        # Extract label from the filename (fake -> 1, real -> 0)
        label = 1 if "fake" in os.path.basename(img_path).lower() else 0

        return img, label, img_path

In [5]:
train_data = CustomImageDataset(directory=f'{PROCESSED_DIR}/{IMAGE_SIZE[0]}/train/rgb')

train_loader = DataLoader(train_data, batch_size=1000, shuffle=True)

# Creating the network, model, training, predicting

In [6]:
network=ClassifierNetwork(IMAGE_SIZE)

In [7]:
model=Model(network, 20)

In [8]:
model.train(train_loader)

Epoch 1/20, Loss: 1.1753672527970735
Epoch 2/20, Loss: 0.6025866880920363
Epoch 3/20, Loss: 0.5447155918393817
Epoch 4/20, Loss: 0.46101849327176253
Epoch 5/20, Loss: 0.36976973869785756
Epoch 6/20, Loss: 0.2761201551241904
Epoch 7/20, Loss: 0.2162919287911113
Epoch 8/20, Loss: 0.16006820884920794
Epoch 9/20, Loss: 0.12146590437207903
Epoch 10/20, Loss: 0.09607401673005235
Epoch 11/20, Loss: 0.07584814220574332
Epoch 12/20, Loss: 0.06036167557102553
Epoch 13/20, Loss: 0.04956029160006076
Epoch 14/20, Loss: 0.04129925461034782
Epoch 15/20, Loss: 0.035872503172212876
Epoch 16/20, Loss: 0.03321911282374622
Epoch 17/20, Loss: 0.02255662088137791
Epoch 18/20, Loss: 0.024761361810407653
Epoch 19/20, Loss: 0.02673747133093406
Epoch 20/20, Loss: 0.01652910099335894


In [ ]:
torch.save(model.state_dict(), 'ADL_model.pth')